In [ ]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [ ]:
champion_mastery = pd.read_csv('../data/champion_mastery.csv')
participants_test = pd.read_csv('../data/participants_test.csv')
participants_train = pd.read_csv('../data/participants_train.csv')
train_winners = pd.read_csv('../data/train_winners.csv')

In [ ]:
champion = pd.read_json('../data/champion.json')

In [ ]:
print(champion.to_string()) 

In [ ]:
pd.DataFrame(champion)

In [ ]:
# Python Parsing JSON 
with open('../data/item.json') as json_file:
    data = json.load(json_file)

print(data)

timeline_*.json

● Contains gameplay information for the first 10 minutes of each match.

● Divided into 11 frames, one starting frame and one for each minute for
the first ten minutes.

○ events: gives details about events taking place in that frame. Eg.
ITEM_PURCHASED, LEVEL_UP, CHAMPION_KILL

○ participantFrames: gives player stats for each participant at the
end of the frame. Eg. health, power, gold, damage done and
damage taken, number of kills, level, xp

In [ ]:
with open('../data/train_timelines/timeline_0.json') as json_file:
    data = json.load(json_file)

print(data)
#pd.DataFrame(data)

In [ ]:
print(data.keys())

In [ ]:
pd.json_normalize(data['frames'])

In [ ]:
pd.json_normalize(data['frames']).columns

#df.columns[df.columns.str.endswith(s)]

In [ ]:
#predictors to get from: gold differential: total gold difference between two participants 

In [ ]:
print(data)

In [ ]:
for i in data['frames']:
    print(i)

In [ ]:
for i in data['frames']:
    print(len(i))

In [ ]:
for i in data['frames']:
    print(i['participantFrames'].keys())

In [ ]:
for i in data['frames']:
    for key, value in i['participantFrames'].items():
        if key == '10':
            print(i['participantFrames'].keys())
    

In [ ]:
type(data['frames'])

In [ ]:
for i in data['frames']:     
    print(i['timestamp'])

In [ ]:
len(data['frames'])

In [ ]:
last_frame = data['frames'][-1]
last_frame

In [ ]:
type(last_frame)

In [ ]:
last_frame.keys()

In [ ]:
type(last_frame['timestamp'])

In [ ]:
#events is a list, participantFrames is a dict,  timestamp is an int
#within events -- info about item purchased, skill_level up ...
#within participantFrames -- info for each participant 1-10

In [ ]:
last_frame['participantFrames'] #stat for participants 1-10 at the lastframe

In [ ]:
type(last_frame['participantFrames'])

In [ ]:
last_frame['participantFrames'].keys()

In [ ]:
for keys, values in last_frame.items():
    print(key)

# ZOOMING INTO PARTICIPANT FRAMES OF THE LAST FRAME

In [ ]:
last_stats = last_frame['participantFrames']
last_stats

In [ ]:
type(last_stats)

In [ ]:
last_stats.keys() #KEYS REPRESENT EACH PARTICIPANT ID

In [ ]:
for i, j in last_stats.items():
    print(i,j['currentGold'], j['totalGold'], j['xp'])

In [ ]:
participantid = []
currentgold = []
totalgold = []
xp = []

for i, j in last_stats.items():
    participantid.append(i)
    currentgold.append(j['currentGold'])
    totalgold.append(j['totalGold'])
    xp.append(j['xp'])

In [ ]:
di = {'participantid' : participantid, 'currentgold': currentgold, 'totalgold':totalgold, 'xp': xp}
z = pd.DataFrame(di)
z

In [ ]:
this also needs to have matchId, teamId

In [ ]:
data['matchId']

In [ ]:
#duplicating above table but start from data:
participantid = []
currentgold = []
totalgold = []
xp = []
matchid = []
teamid = []

    
for i, j in data['frames'][-1]['participantFrames'].items():  #looks at last frame, looks at participantFrames equiv to last_stats
    participantid.append(i)
    currentgold.append(j['currentGold'])
    totalgold.append(j['totalGold'])
    xp.append(j['xp'])
    matchid.append(data['matchId'])
    if int(i) < 6:
        teamid.append('100')
    else:
        teamid.append('200')
        
    
  

In [ ]:
di = {'matchid':matchid, 'teamid':teamid, 'participantid' : participantid, 'currentgold': currentgold, 'totalgold':totalgold, 'xp': xp}
z = pd.DataFrame(di)
z

# Scale to all train_timelines files

In [ ]:
import os
import glob

In [ ]:
folder_path = '../data/train_timelines/'
file_list = glob.glob(os.path.join(folder_path, '*.json'))

In [ ]:
file_list

In [ ]:
participantid = []
currentgold = []
totalgold = []
xp = []
matchid = []
teamid = []


def my_code(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
    matchida= data['matchId']
    for i, j in data['frames'][-1]['participantFrames'].items():  #looks at last frame, looks at participantFrames equiv to last_stats
        participantid.append(i)
        currentgold.append(j['currentGold'])
        totalgold.append(j['totalGold'])
        xp.append(j['xp'])
        matchid.append(matchida)
        if int(i) < 6:
            teamid.append('100')
        else:
            teamid.append('200')
        
    
   


In [ ]:
count = 0
for file in file_list:
    #print('loading file: ' + str(count))
    my_code(file)
    count+=1
print("done with loading files.. " + str(count) + " files loaded")

In [ ]:
di = {'matchid':matchid, 'teamid':teamid, 'participantid' : participantid, 'currentgold': currentgold, 'totalgold':totalgold, 'xp': xp}
z = pd.DataFrame(di)
z

In [ ]:
z.head(20)

In [ ]:
z['matchid'].nunique()

In [ ]:
gold_data = pd.DataFrame(z.groupby(['matchid', 'teamid'])['currentgold'].sum()).reset_index()
gold_data

In [ ]:
train_winners.head()

In [ ]:
gold_winner = pd.merge(gold_data, train_winners, left_on='matchid', right_on='matchId').drop('matchid', axis = 1)

In [ ]:
gold_winner.head(10)